In [1]:
import numpy as np
from data import X_train, X_test, y_train, y_test
from random_forest import RandomForestList,PredictRandomForest
from tree import print_tree


In [2]:
random_forest_list, X_sample_list = RandomForestList(X_train, y_train, n_trees=1, max_depth=5, min_samples_split=5, min_samples_leaf=2, max_features=30, bootsrap=False)

In [3]:
print(random_forest_list.__len__())
print(len(X_sample_list))

1
1


In [4]:
for i in random_forest_list:
    print_tree(i.tree)

Node d=0 samples=455 gini=0.4635 pred=0 feat=28 thr=0.14235
  Node d=1 samples=305 gini=0.1505 pred=0 feat=12 thr=16.805
    Node d=2 samples=278 gini=0.0491 pred=0 feat=None thr=None
    Node d=2 samples=27 gini=0.4444 pred=1 feat=4 thr=16.375
      Node d=3 samples=7 gini=0.0000 pred=0 feat=None thr=None
      Node d=3 samples=20 gini=0.1800 pred=1 feat=None thr=None
  Node d=1 samples=150 gini=0.1128 pred=1 feat=13 thr=13.93
    Node d=2 samples=3 gini=0.0000 pred=0 feat=None thr=None
    Node d=2 samples=147 gini=0.0783 pred=1 feat=1 thr=0.012910000000000001
      Node d=3 samples=145 gini=0.0537 pred=1 feat=2 thr=0.055650000000000005
        Node d=4 samples=10 gini=0.4800 pred=1 feat=25 thr=19.615000000000002
          Node d=5 samples=4 gini=0.0000 pred=0 feat=None thr=None
          Node d=5 samples=6 gini=0.0000 pred=1 feat=None thr=None
        Node d=4 samples=135 gini=0.0000 pred=1 feat=None thr=None
      Node d=3 samples=2 gini=0.0000 pred=0 feat=None thr=None


In [5]:
y_pred = PredictRandomForest(random_forest_list, X_sample_list)
accuracy = np.mean(y_pred == y_train) 
print("Accuracy (en train):", accuracy)

Accuracy (en train): 0.6351648351648351
